In [7]:
# import skimage.measure as msr
import numpy as np
# import matplotlib
# import matplotlib.pyplot as plt
# import matplotlib.style as style
# import pandas as pd
# from PIL import Image, ImageCms

import glob
import os
from pathlib import Path
# import imageio
import cv2

In [8]:
# Importing class from wmark.py
from wmark import WaterMark

ModuleNotFoundError: No module named 'skimage'

In [ ]:
# Source Directory
src_folder = 'TestSet/'
# Source Path
src_pth = Path(src_folder).resolve()

# Destination Directory
dst_folder = Path(src_folder+'/marked/').resolve()
# Creating folder for marked images
Path(dst_folder).mkdir(exist_ok=True)
# Destination Path
dst_path = Path(dst_folder).resolve()

In [ ]:
# Creating empty DataFrames that will fill up during the loop
big_corr = pd.DataFrame()

for i in range(1, 2):
    
    # Generating random seed as integer up to 10000
    random_nums = np.random.randint(10000, size=(1))
    random_num = random_nums[0]
    print(random_num)

    w_object = WaterMark(random_num)

    # All TIF files in the src_path are now imgs
    imgs_source = src_pth.glob('*.tif')
    
    # Creating empty DataFrames that will fill up during the loop
    final_corr_original = pd.DataFrame()
    
    # Creating empty DataFrames that will fill up during the loop
    final_corr_marked = pd.DataFrame()
    
    # Counting the number of images in src_folder
    list = os.listdir(src_folder)
    # Counting folder as two files?!
    num_images = len(list) - 2
    
    results = np.zeros([num_images, 9])
    counter = 0
        

    for img in imgs_source:
    
        # Read original image
        img_original = WaterMark.imread(img)
        
        # Find impact factor within a PSNR range
        find_impact_factor = w_object.findImpactFactor(img_original, rangePSNR = (35,40))
        
        # Mark image
        img_marked = w_object.embedMark(img_original, factor = find_impact_factor)
        # img_name_io = f"{str(dst_folder)}/marked-{random_num}-{str(img.name)}"
        # imageio.imwrite(img_name_io, img_to_dst)
        
        
        #CROPPING
        #CROPPING
        #CROPPING
        
        left, top, right, bottom = 0, 0, 45, 45
        img_cropped = img_marked.crop((left, top, right, bottom))
        
        
        #SCALING
        #SCALING
        #SCALING
        
        # Wanted value of scaled image (in pixels)
        height = 768
        width = 768
        # Scaling
        img_scaled = cv2.resize(img_marked, (width, height))

        
        #ROTATION
        #ROTATION
        #ROTATION
        
        angle = 90
        rows, cols = img_marked.shape[:2]
        # Defines center if image and angle of rotation
        M = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
        # Defines result image
        img_rotated = cv2.warpAffine(img_marked, M, (cols, rows))    

        
        
        # AFFINE TRANSFORMATION
        # AFFINE TRANSFORMATION
        # AFFINE TRANSFORMATION
        
        # Defining points in original image
        point_1_a = [50, 50]
        point_1_b = [200, 50]
        point_1_c = [50, 200]
        # Defining points in transformed image
        point_2_a = [10, 100]
        point_2_b = [200, 50]
        point_2_c = [100, 250]
        
        rows, cols, ch = img_marked.shape
        # Compiling original points into single array
        pts1 = np.float32([point_1_a, point_1_b, point_1_c])
        # Compiling transformed points into single array
        pts2 = np.float32([point_2_a, point_2_b, point_2_c])
        M = cv2.getAffineTransform(pts1, pts2)
        img_affined = cv2.warpAffine(img_marked, M, (cols, rows))
       
        
        # 2D CONVOLUTION
        # 2D CONVOLUTION
        # 2D CONVOLUTION
        
        # Horizontal deviation (intensity)
        kernel_x = 3
        # Vertical deviation (intensity)
        kernel_y = 3
        kernel_x_str = str(kernel_x)
        kernel_y_str = str(kernel_y)
        kernel_multiplication = kernel_x * kernel_y
        
        kernel = np.ones((kernel_x, kernel_y), np.float32)/kernel_multiplication
        img_convoluted = cv2.filter2D(img_marked, -1, kernel)
       
        
        # GAUSSIAN BLUR
        # GAUSSIAN BLUR
        # GAUSSIAN BLUR
        
        # Horizontal deviation (blur intensity)
        kernel_x = 3
        # Vertical deviation (blur intensity)
        kernel_y = 3
        kernel_x_str = str(kernel_x)
        kernel_y_str = str(kernel_y)
        
        img_blurred = cv2.GaussianBlur(img_marked, (kernel_x, kernel_y), cv2.BORDER_DEFAULT)
  
        
        # GAUSSIAN NOISE
        # GAUSSIAN NOISE
        # GAUSSIAN NOISE
        
        # Wanted value of rotation angle (in degrees)
        noise_amount = 0.5
        
        # Generate Gaussian noise
        gauss = np.random.normal(0, noise_amount, img_marked.size)
        gauss = gauss.reshape(
            img_marked.shape[0], img_marked.shape[1], img_marked.shape[2]).astype('uint8')
        # Add the Gaussian noise to the image
        img_noised = cv2.add(img_marked, gauss)
      
        
        corr_original = w_object.decodeMark(img_original)
        results[counter, 0] = corr_original
        
        corr_marked = w_object.decodeMark(img_marked)
        results[counter, 1] = corr_marked
        
        corr_scaled = w_object.decodeMark(img_scaled)
        results[counter, 2] = corr_scaled
        
        corr_rotated = w_object.decodeMark(img_rotated)
        results[counter, 3] = corr_rotated
        
        corr_affined = w_object.decodeMark(img_affined)
        results[counter, 4] = corr_affined
        
        corr_convoluted = w_object.decodeMark(img_convoluted)
        results[counter, 5] = corr_convoluted
        
        corr_blurred = w_object.decodeMark(img_blurred)
        results[counter, 6] = corr_blurred
        
        corr_noised = w_object.decodeMark(img_noised)
        results[counter, 7] = corr_noised
        
        counter += 1
        
        
        pandas_values = ["Cropped", "Original", "Marked", "Scaled", "Rotated", "Affined", "2D Convoluted", "Blurred", "Noised"]
        pandas_seeds = [f"{random_num}", f"{random_num}", f"{random_num}", f"{random_num}", f"{random_num}", f"{random_num}", f"{random_num}", f"{random_num}", f"{random_num}"]
        
        final_corr = pd.DataFrame(results, columns = [pandas_seeds, pandas_values])
    

    big_corr = pd.concat([big_corr, final_corr], axis = 1)

In [ ]:
plt.imshow(img_noised)

In [ ]:
# Show Pandas DataFrame
big_corr

In [ ]:
stacked_dataframe = big_corr.stack(0)

In [ ]:
stacked_dataframe

In [ ]:
big_corr.describe()

In [ ]:
plt.style.use('fivethirtyeight')

stacked_dataframe.plot(kind = 'bar', stacked = False, legend = True)
plt.show()

In [ ]:
stacked_dataframe.plot.hist(legend = True)